# Lab 5

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zyang91/spatial_data_management/blob/main/lab/lab_05.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

In [ ]:
import geemap
import ee

ee.Initialize()

## Question 1

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

In [ ]:
m = geemap.Map(center=[40, -100], zoom=4)
collection = ee.FeatureCollection("USGS/WBD/2017/HUC04")
style = {"color": "ff0000ff", "width": 2, "lineType": "solid", "fillColor": "00000000"}
m.addLayer(collection.style(**style), {}, "HUC04")
m

In [ ]:
text = "Made by Zhanchao Yang"
m.add_text(text, fontsize=20, position="bottomright")

![](https://i.imgur.com/PLlNFq3.png)

## Question 2 

Filter the USGS Watershed Boundary dataset and select the watershed that intersects the county of your choice.

In [ ]:
fc = ee.FeatureCollection("TIGER/2018/Counties")
subset = fc.filter(ee.Filter.eq("NAME", "Knox"))
knox = subset.filter(ee.Filter.eq("STATEFP", "47"))
style = {"color": "0000ffff", "width": 2, "lineType": "solid", "fillColor": "00000000"}
m.addLayer(knox.style(**style), {}, "Knox County, TN")
m.center_object(knox, zoom=8)

In [ ]:
bound = knox
selected_fc = ee.FeatureCollection("USGS/WBD/2017/HUC04").filterBounds(bound)

style = {"color": "ff0000ff", "width": 2, "lineType": "solid", "fillColor": "00000050"}
m.addLayer(selected_fc.style(**style), {}, "HUC04 in Knox County, TN")
m.center_object(selected_fc, zoom=10)
m

![](https://i.imgur.com/F2QfqZu.png)

## Question 3

Clip the [USGS 3DEP 10m DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m) with the watershed that intersects the county of your choice. Display the DEM with a proper color palette and color bar.

In [ ]:
image = ee.Image("USGS/3DEP/10m")
subset = image.clip(selected_fc)
vis_params = {
    "min": 0,
    "max": 1500,
    "palette": "terrain",
}
m.addLayer(subset, vis_params, "USGS/3DEP/10m")
m

In [ ]:
m.add_colorbar(vis_params, label="Elevation (m)")

![](https://i.imgur.com/okR39pf.png)

## Question 4

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [ ]:
states = ee.FeatureCollection("TIGER/2018/States")
New_york = states.filter(ee.Filter.eq("NAME", "New York"))
m = geemap.Map(center=[43.2994, -74.2179], zoom=6)
style = {"color": "0000ffff", "width": 2, "lineType": "solid", "fillColor": "00000000"}
m.addLayer(New_york.style(**style), {}, "New York State")
m

In [ ]:
nlcd_2001 = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2001").select("landcover")
nlcd_2019 = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select("landcover")

subset_2001 = nlcd_2001.clip(New_york)
subset_2019 = nlcd_2019.clip(New_york)

left_layer = geemap.ee_tile_layer(subset_2001, {}, "NLCD 2001")
right_layer = geemap.ee_tile_layer(subset_2019, {}, "NLCD 2019")

m.split_map(left_layer, right_layer, add_close_button=True)
m

In [ ]:
legend_dict = {
    "11 Open Water": "466b9f",
    "12 Perennial Ice/Snow": "d1def8",
    "21 Developed, Open Space": "dec5c5",
    "22 Developed, Low Intensity": "d99282",
    "23 Developed, Medium Intensity": "eb0000",
    "24 Developed High Intensity": "ab0000",
    "31 Barren Land (Rock/Sand/Clay)": "b3ac9f",
    "41 Deciduous Forest": "68ab5f",
    "42 Evergreen Forest": "1c5f2c",
    "43 Mixed Forest": "b5c58f",
    "51 Dwarf Scrub": "af963c",
    "52 Shrub/Scrub": "ccb879",
    "71 Grassland/Herbaceous": "dfdfc2",
    "72 Sedge/Herbaceous": "d1d182",
    "73 Lichens": "a3cc51",
    "74 Moss": "82ba9e",
    "81 Pasture/Hay": "dcd939",
    "82 Cultivated Crops": "ab6c28",
    "90 Woody Wetlands": "b8d9eb",
    "95 Emergent Herbaceous Wetlands": "6c9fb8",
}
m.add_legend(title="NLCD Land Cover Classification", legend_dict=legend_dict)

In [ ]:
text = "Made by Zhanchao Yang"
m.add_text(text, fontsize=13, position="bottomright")

In [ ]:
m.save("NewYork.html")

![](https://i.imgur.com/Au7Q5Ln.png)

## Questions 5

Download OpenStreetMap data for a city of your choice and visualize the city boundary and restaurants in the city.

In [ ]:
import geemap.osm as osm

In [ ]:
Map = geemap.Map()
binghamton = geemap.osm_to_ee("Binghamton, New York")
Map.addLayer(binghamton, {}, "Binghamton")
Map.centerObject(binghamton, 12)
Map

In [ ]:
place = "Binghamton, New York"
tags = {"amenity": "restaurant"}
gdf = osm.osm_gdf_from_place(place, tags)
Map.add_gdf(gdf, "Restaurants in Binghamton, New York")

In [ ]:
text = "Made by Zhanchao Yang"
Map.add_text(text, fontsize=20, position="bottomright")

![](https://i.imgur.com/AUlO1CV.png)